In [1]:
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import html
import re
from sklearn.feature_extraction.text import strip_accents_ascii, strip_accents_unicode
import matplotlib.pyplot as plt
import math

%matplotlib inline
numcores = 16
tiene_gpu = False
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 100)

import os 


In [2]:
# Este es el root en el servidor de jupyter
data_root = 'C:/code/hotelmapping/data/'


## Deshabilito los future wanings ya que se resolverá en el futuro

In [3]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


### Cargo datos

In [4]:
providers_clean_file =os.path.join(data_root, 'travcoding/Providers_clean.parquet')

# Cargo los archivos
inventory_ddf = dd.read_parquet(properties_clean_file)
provider_ddf  = dd.read_parquet(providers_clean_file)


### Genero las estadisticas de las palabras de la columna del nombre de la propiedad y de cada registro

In [5]:

def get_word_stats(ddf, colum_name,key_name):            
    ddf1 = ddf[colum_name].str.split().explode()
    ddf2 = ddf1.value_counts().reset_index().rename(columns={'index': 'word',colum_name:'cantidad'})                           
    ddf2['freq'] = (ddf2.cantidad/ddf2.cantidad.sum())
    ddf2['idf']  = (da.log(1/ddf2.freq))
    ddf3= ddf1.to_frame().reset_index().merge(ddf2, how='left', left_on=colum_name, right_on='word')[[key_name,'idf']]
    ddf4=ddf3.groupby(by=[key_name]).agg({'idf': ['count','sum','max','mean'] }).reset_index()
    ddf4.columns = ["_".join(x) for x in ddf4.columns.ravel()]
    ddf4 = ddf4.rename(columns={(key_name+'_'): key_name}).set_index(key_name)    
    return ddf2, ddf4


In [6]:
column_name = 'propertyname'
keycol_name = 'PropertyId'
inventory_words_stats_ddf, inventory_propertyname_stats_ddf = get_word_stats(inventory_ddf, column_name, keycol_name)


In [7]:
#inventory_words_stats_ddf.head()
inventory_propertyname_stats_ddf.head()

,idf_count,idf_sum,idf_max,idf_mean
PropertyId,,,,
207,3,15.768009,8.890674,5.256003
860,5,47.778503,11.855947,9.555701
1181,3,21.143756,9.452010,7.047919
1287,4,30.184631,9.666158,7.546158
1393,6,38.653313,11.196702,6.442219


In [8]:
column_name = 'propertyname'
keycol_name = 'PropertyByProviderId'
provider_words_stats_ddf, provider_propertyname_stats_ddf = get_word_stats(provider_ddf, column_name, keycol_name)


### Genera Stopwords y Regex de stopwords 

In [9]:

# Devuelve una lista con los stopwords a partir del df de estadisticas
def get_stopwords_list(word_stats_ddf,cantidad_minima):
    stopwords = []
    for stopword in word_stats_ddf[word_stats_ddf.cantidad>cantidad_minima].word:
        stopwords.append(stopword.rstrip('\n').lower())    
    return stopwords

In [10]:
cantidad_minima = 1000
property_name_stopwords = get_stopwords_list(inventory_words_stats_ddf,cantidad_minima)
property_name_stopwords_regex = re.compile(r'\b(' + r'|'.join(property_name_stopwords) + r')\b')


In [11]:
blocking_column = 'propertyname'



cant_palabras_total = len(inventory_words_stats_ddf.index)
cant_palabras_bloqueadas = len(property_name_stopwords)
print("cant_palabras_total=", cant_palabras_total, " cant_palabras_bloqueadas=", cant_palabras_bloqueadas)


#Save Stopwords
#Inventory_words_stats[Inventory_words_stats.cantidad>cantidad_minima].compute().sort_values('cantidad', ascending=False).to_csv('data/travcoding/stopwords.csv') 


cant_palabras_total= 193628  cant_palabras_bloqueadas= 210


# Blocking


In [12]:
def get_blocking_df( data_ddf, 
                blocking_column, 
                stopwords_regex, 
                aditional_blocking_column='countrycorregido'):       
    return data_ddf[blocking_column].str.replace(stopwords_regex,'').str.split().explode().dropna()\
        .to_frame().rename(columns={ blocking_column :  'value'})\
        .merge(data_ddf, left_index=True, right_index=True)[['value', aditional_blocking_column]].drop_duplicates().reset_index()
        #.compute()
    

    

In [13]:
blocking_column = 'propertyname'    

### Genera indice para el inventario

In [14]:
inv_indx_ddf = get_blocking_df(inventory_ddf,blocking_column,property_name_stopwords_regex)
#inv_indx = inv_indx.reset_index()
print("Index rows = ", inv_indx_ddf.index.size.compute())
inv_indx_ddf.head(3)

Index rows =  326139


,PropertyId,value,countrycorregido
0,1181,expo,spain
1,1181,barcelona,spain
2,7633,coralinn,united kingdom


### Genera indice para proveedores

In [15]:
prv_indx_ddf = get_blocking_df(provider_ddf,blocking_column,property_name_stopwords_regex)
print("Index rows = ", prv_indx_ddf.index.size.compute())
prv_indx_ddf.head(3)



Index rows =  445457


,PropertyByProviderId,value,countrycorregido
0,9,heliotrope,greece
1,19,monroe,united states
2,19,unit,united states


### Genera candidatos

In [17]:

candidates_ddf = dd.merge(inv_indx_ddf, prv_indx_ddf, how='inner', left_on=['value','countrycorregido'], right_on=['value','countrycorregido'])
print("Index rows = ", candidates_ddf.index.size.compute())
# candidates_ddf = dd.from_pandas(candidates_ddf, npartitions=numcores)
candidates_ddf.head(3)


Index rows =  325873


,PropertyId,value,countrycorregido,PropertyByProviderId
0,1181,expo,spain,1583630
1,1181,barcelona,spain,59555
2,7633,coralinn,united kingdom,13283


In [18]:
candidates_ddf = candidates_ddf.merge(inventory_words_stats_ddf, how='left', left_on='value', right_on='word')
candidates_ddf.head()

,PropertyId,value,countrycorregido,PropertyByProviderId,word,cantidad,freq,idf
0,1181,expo,spain,1583630,expo,166,7.853159e-05,9.452010
1,1181,barcelona,spain,59555,barcelona,288,1.362476e-04,8.901037
2,7633,coralinn,united kingdom,13283,coralinn,2,9.461638e-07,13.870850
3,9704,verde,south africa,2545731,verde,413,1.953828e-04,8.540550
4,9704,cape,south africa,28940,cape,269,1.272590e-04,8.969286


In [27]:
# Agrego candidad de palabras y suma de IDF
pair_candidates2 = candidates_ddf.groupby(['PropertyId','PropertyByProviderId']).agg({'idf': ['count','sum','max','mean'] }).reset_index()

#Aplano los niveles del multiindex
pair_candidates2.columns = ["_".join([z.strip() for z in x if z.strip()]) for x in pair_candidates2.columns.ravel()]




In [40]:
# File path a los archivos
pair_candidates_file = os.path.join(data_root, 'travcoding/pair_candidates.parquet')    
pair_candidates_file
pair_candidates2.to_parquet(pair_candidates_file)

In [28]:
pair_candidates2.head()

,PropertyId,PropertyByProviderId,idf_count,idf_sum,idf_max,idf_mean
0,1181,59555,1,8.901037,8.901037,8.901037
1,1181,1583630,1,9.452010,9.452010,9.452010
2,1490,360,1,13.870850,13.870850,13.870850
3,2270,1166709,1,13.870850,13.870850,13.870850
4,5103,1223830,1,12.079091,12.079091,12.079091


In [30]:
candidates = pair_candidates2.merge(inventory_ddf, how='inner', left_on=['PropertyId'], right_index=True)\
    .merge(provider_ddf, how='inner', left_on=['PropertyByProviderId'], right_index=True)


In [34]:
cols = ['PropertyId',
    'PropertyByProviderId',
    'idf_count',
    'idf_sum',
    'idf_max',
    'idf_mean',
    #---------------------------------------
    'countrycode_x',
    'countrycode_y',
    'countrycorregido_x',
    'countrycorregido_y',
    'propertytype_x',
    'propertytype_y',
    'propertyname_x',
    'propertyname_y',
    'lat_x',
    'lng_x',
    'lat_y',
    'lng_y',
    'address_x',
    'address_y',
    'zipcode_y',
    'zipcode_x',
    'city_x',
    'city_y',
    'state_y',
    'state_x',
    'starrating_y',
    'starrating_x',
    'email_x',
    'email_y',
    'phone_x',
    'phone_y',
    'fax_y',
    'fax_x',
    'website_x',
    'website_y'
    # 'providerid',
    # 'propertyid',
    #'language'
    ]

In [38]:
candidates.head()[cols]

,PropertyId,PropertyByProviderId,idf_count,idf_sum,idf_max,idf_mean,countrycode_x,countrycode_y,countrycorregido_x,countrycorregido_y,propertytype_x,propertytype_y,propertyname_x,propertyname_y,lat_x,lng_x,lat_y,lng_y,address_x,address_y,zipcode_y,zipcode_x,city_x,city_y,state_y,state_x,starrating_y,starrating_x,email_x,email_y,phone_x,phone_y,fax_y,fax_x,website_x,website_y
221814,667295,278914,1,13.465385,13.465385,13.465385,MY,MY,malaysia,malaysia,hotel,hotel,oyo 778 bmw 2 hotel,oyo 90155 bmw 1 hotel,2.067920,102.602364,2.064004,102.592438,1 & 2 jalan terap 1 taman seri terap,no 18 1 taman indah jalan haji abdullah,84000,84000,Muar,Muar,johor,,2,2,,,6072687407,6072687407,,,,
158083,549171,278914,1,14.563997,14.563997,14.563997,MY,MY,malaysia,malaysia,hotel,hotel,oyo 90155 bmw 1 hotel,oyo 90155 bmw 1 hotel,2.064004,102.592438,2.064004,102.592438,no 18 1 taman indah jalan haji abdullah,no 18 1 taman indah jalan haji abdullah,84000,84000,Muar,Muar,johor,johor,2,2,,,6072687407,6072687407,,,,
221819,667298,397041,2,28.434848,14.563997,14.217424,KR,KR,south korea,south korea,hotel,private vacation home,i pluss poolviller pension,i pluss poolviller pension,37.881321,127.730003,37.881321,127.730003,22 1 pulmugol 1 gil namsan myeon,22 1 pulmugol 1 gil namsan myeon,24465,24465,Chuncheon,Chuncheon,,,3,3,,,821050298900,821050298900,,,,
221821,667304,195427,1,13.870850,13.870850,13.870850,ID,ID,indonesia,indonesia,hotel,hotel,reddoorz plus jalan dr cipto mangunkusumo lampung,penginapan cipto wening,-5.426362,105.270386,-7.607609,110.427238,jl. ir. h juanda/kesehatan,jl. kaliurang no.24 ngipiksari,55582,<NA>,Bandar Lampung,Pakem,special region of yogyakarta,lampung,2,2,,,622880629666,6281227957561,,,,
111168,465684,195427,1,14.563997,14.563997,14.563997,ID,ID,indonesia,indonesia,hotel,hotel,penginapan cipto wening,penginapan cipto wening,-7.607609,110.427238,-7.607609,110.427238,jl. kaliurang no.24 ngipiksari,jl. kaliurang no.24 ngipiksari,55582,55582,Pakem,Pakem,special region of yogyakarta,special region of yogyakarta,2,2,,,6281227957561,6281227957561,,,,
